In [0]:
import os
import requests
import json
from tqdm.auto import tqdm
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, size

In [0]:
DBFS_DIR = "/Volumes/wine_harmonization/datasets/raw_datasets/flavor_profiles_v4"
if not os.path.exists(DBFS_DIR):
    os.makedirs(DBFS_DIR)

BASE_URL = "https://cosylab.iiitd.edu.in/flavordb2"
ENTITY_URL = BASE_URL + "/entities_json?id={}"

In [0]:
def download_flavordb_data(max_id=10000, start_id=1):
    """
    Baixa dados do FlavorDB2 usando o endpoint entities_json
    
    Args:
        max_id: ID máximo para tentar baixar
        start_id: ID inicial para começar o download
    """
    
    successful_downloads = 0
    failed_downloads = 0
    
    print(f"Iniciando download do FlavorDB2 (IDs {start_id} a {max_id})")
    
    for entity_id in tqdm(range(start_id, max_id + 1), desc="Baixando dados"):
        try:
            # Construir URL
            url = ENTITY_URL.format(entity_id)
            
            # Verificar se arquivo já existe
            file_name = os.path.join(DBFS_DIR, f"{entity_id}.json")
            if os.path.exists(file_name) and os.path.getsize(file_name) > 0:
                successful_downloads += 1
                continue
            
            # Fazer requisição
            response = requests.get(url, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                
                # Verificar se os dados são válidos
                if data and isinstance(data, dict):
                    # Tentar obter nome da entidade para log
                    name_keys = ["entity_alias_readable", "name", "entity_name", "title"]
                    entity_name = str(entity_id)
                    
                    for key in name_keys:
                        if key in data and data[key]:
                            entity_name = data[key]
                            break
                    
                    # Salvar arquivo
                    with open(file_name, 'w', encoding='utf-8') as f:
                        json.dump(data, f, indent=4, sort_keys=True, ensure_ascii=False)
                    
                    successful_downloads += 1
                    
                    # Log a cada 100 downloads bem-sucedidos
                    if successful_downloads % 100 == 0:
                        print(f"✓ {successful_downloads} downloads concluídos - Último: {entity_name}")
                
                else:
                    failed_downloads += 1
                    
            else:
                failed_downloads += 1
                
        except requests.exceptions.RequestException as e:
            failed_downloads += 1
            if entity_id % 1000 == 0:  # Log erros apenas a cada 1000
                print(f"Erro de rede no ID {entity_id}: {e}")
                
        except json.JSONDecodeError:
            failed_downloads += 1
            if entity_id % 1000 == 0:
                print(f"Erro JSON no ID {entity_id}")
                
        except Exception as e:
            failed_downloads += 1
            if entity_id % 1000 == 0:
                print(f"Erro geral no ID {entity_id}: {e}")
        
        # Pequeno delay para não sobrecarregar o servidor
        if entity_id % 10 == 0:
            time.sleep(0.1)
    
    return successful_downloads, failed_downloads

def check_existing_files():
    """Verifica arquivos já baixados"""
    if os.path.exists(DBFS_DIR):
        files = [f for f in os.listdir(DBFS_DIR) if f.endswith('.json')]
        print(f"Arquivos já existentes: {len(files)}")
        
        if files:
            # Mostrar alguns exemplos
            example_files = sorted(files)[:5]
            print("Exemplos de arquivos:", example_files)
            
            # Encontrar maior ID já baixado
            ids = []
            for f in files:
                try:
                    file_id = int(f.replace('.json', ''))
                    ids.append(file_id)
                except ValueError:
                    continue
            
            if ids:
                max_existing = max(ids)
                print(f"Maior ID já baixado: {max_existing}")
                return max_existing
    
    return 0

def sample_data():
    """Mostra uma amostra dos dados baixados"""
    files = [f for f in os.listdir(DBFS_DIR) if f.endswith('.json')]
    if files:
        sample_file = os.path.join(DBFS_DIR, files[0])
        with open(sample_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print(f"\n=== Amostra dos dados (arquivo: {files[0]}) ===")
        print(f"Chaves disponíveis: {list(data.keys())}")
        
        # Mostrar algumas chaves importantes
        important_keys = ["entity_alias_readable", "name", "entity_name", "category", "flavor_profile"]
        for key in important_keys:
            if key in data:
                print(f"{key}: {data[key]}")

if __name__ == "__main__":
    print("=== FlavorDB2 Downloader ===")
    
    # Verificar arquivos existentes
    max_existing = check_existing_files()
    
    # Definir range de IDs para baixar
    START_ID = max_existing + 1 if max_existing > 0 else 1
    MAX_ID = 1000  # Ajuste conforme necessário
    
    print(f"Iniciando download a partir do ID {START_ID}")
    
    # Executar download
    successful, failed = download_flavordb_data(MAX_ID, START_ID)
    
    print(f"\n=== Resultados Finais ===")
    print(f"Downloads bem-sucedidos: {successful}")
    print(f"Downloads falharam: {failed}")
    print(f"Arquivos salvos em: {DBFS_DIR}")
    
    # Mostrar amostra dos dados
    sample_data()
    
    print("\nDownload concluído!")